<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/chat_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import sqlite3
from tabulate import tabulate
import datetime

db = 'chat.db'

def show_tables():
  tables = ['users', 'chats', 'messages']
  def show(tablename):
    try:
      with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        cursor.execute(f'SELECT * FROM {tablename}')
        results = cursor.fetchall()
        headers = []
        for i in range(len(cursor.description)):
          headers.append(cursor.description[i][0])
        print(tablename)
        print(tabulate(results, headers=headers, tablefmt='grid', stralign='center'))
        print("\n")
    except:
      print("Empty")

  for table in tables:
    show(table)

show_tables()

users
+------+------------+
|   id |  username  |
+======+============+
|    1 |    Ivan    |
+------+------------+
|    2 |    Kate    |
+------+------------+


chats
+------+-----------+------------+---------------------+
|   id |   user_id |  chatname  |     created_at      |
+======+===========+============+=====================+
|    3 |         1 |  My Chat   | 2023-11-13 04:07:04 |
+------+-----------+------------+---------------------+


messages
+------+-----------+-----------+-----------+--------------+
|  id  |  user_id  |  chat_id  |  message  |  created_at  |
+======+===========+===========+===========+==============+
+------+-----------+-----------+-----------+--------------+




In [54]:
def create_db():
  with sqlite3.connect(db) as connection:
    cursor = connection.cursor()
    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT NOT NULL
        )
        """
    )
    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS chats (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER NOT NULL,
            chatname TEXT NOT NULL,
            created_at DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (user_id) REFERENCES users(id) ON DELETE CASCADE
        )
        """
    )
    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS messages (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER NOT NULL,
            chat_id INTEGER NOT NULL,
            message TEXT NOT NULL,
            created_at DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (user_id) REFERENCES users(id) ON DELETE CASCADE,
            FOREIGN KEY (chat_id) REFERENCES chats(id) ON DELETE CASCADE
        )
        """
    )
    connection.commit()



def add_user(username):
    with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        user = cursor.execute("SELECT id FROM users WHERE username = ?", (username,)).fetchone()
        if user is not None:
            print("Пользователь существует")
            return
        cursor.execute("INSERT INTO users (username) VALUES (?)", (username,))
        connection.commit()


def add_chat(username, chatname):
  with sqlite3.connect(db) as connection:
    cursor = connection.cursor()
    chat = cursor.execute("SELECT id FROM chats WHERE chatname = ?", (chatname,)).fetchone()
    if chat is not None:
        print("Чат существует")
        return
    user_id = cursor.execute("SELECT id FROM users WHERE username = ?", (username,)).fetchone()[0]
    cursor.execute("INSERT INTO chats (user_id, chatname) VALUES (?, ?)", (user_id, chatname,))
    connection.commit()


def add_message(username, chatname, message):
  with sqlite3.connect(db) as connection:
    cursor = connection.cursor()
    user_id = cursor.execute("SELECT id FROM users WHERE username = ?", (username,)).fetchone()[0]
    chat_id = cursor.execute("SELECT id FROM chats WHERE chatname = ?", (chatname,)).fetchone()[0]
    cursor.execute("INSERT INTO messages (user_id, chat_id, message) VALUES (?, ?, ?)", (user_id, chat_id, message,))
    connection.commit()


def delete_user(username):
  with sqlite3.connect(db) as connection:
    cursor = connection.cursor()
    cursor.execute("DELETE FROM users WHERE username = ?", (username,))
    connection.commit()


def delete_chat(username, chatname):
  with sqlite3.connect(db) as connection:
    cursor = connection.cursor()
    user_id = cursor.execute("SELECT id FROM users WHERE username = ?", (username,)).fetchone()[0]
    chat_id = cursor.execute("SELECT id FROM chats WHERE chatname = ?", (chatname,)).fetchone()[0]
    cursor.execute("DELETE FROM messages WHERE chat_id = ?", (chat_id,))
    cursor.execute("DELETE FROM chats WHERE id = ?", (chat_id,))
    connection.commit()


def delete_message(username, chatname, message):
  with sqlite3.connect(db) as connection:
    cursor = connection.cursor()
    user_id = cursor.execute("SELECT id FROM users WHERE username = ?", (username,)).fetchone()[0]
    chat_id = cursor.execute("SELECT id FROM chats WHERE chatname = ?", (chatname,)).fetchone()[0]
    cursor.execute("DELETE FROM messages WHERE user_id = ? AND chat_id = ? AND message = ?", (user_id, chat_id, message,))
    connection.commit()


create_db()

# add_user("Ivan")
# add_chat("Ivan", "My Chat")
# add_message("Ivan", "My Chat", "Hello, world!")
# add_user("Kate")
# add_chat("Kate", "My Chat")
# add_message("Kate", "My Chat", "Hello, world!")
# delete_user("Ivan")
# delete_chat("Ivan", "My Chat")
# delete_message("Ivan", "My Chat", "Hello, world!")
# delete_user("Kate")
# delete_chat("Kate", "My Chat")
# delete_message("Kate", "My Chat", "Hello, world!")